In [1]:
import config
import mysql.connector
from IPython.display import display, Markdown
import csv
import os

### Helper Functions

In [2]:
def batch_execute_ddl(conn, file):
    cursor = conn.cursor()
    ddl_file = open(file)
    sql = ddl_file.read()
    
    # print(sql)
    # multi = True for multple statements
    for result in cursor.execute(sql, multi=True):
        if result.with_rows:
            print(f"Rows produced: {result.statement}")
            print(result.fetchall())
        else: 
            print(f"number of rows affected by statement {result.statement}: {result.rowcount}")
    
    conn.close()
    
def insert_data(command, data, conn):
    try:
        # Connecting to the Database 
        # print(data)
        cursor = conn.cursor()
        cursor.executemany(command, data) 
        
        conn.commit()
        print("Rows affected: ",cursor.rowcount, "\nData inserted successfully")

    except mysql.connector.Error as error:
        print(error)
        conn.rollback()

        
def fetch(query, conn):
    cursor = conn.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    return result
    # print("Result:", result)

def update(query, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        result = cursor.fetchall()
        # print("Result:", result)111
        
        conn.commit()
    except mysql.connector.Error as error:
        print(error)
        conn.rollback()
        
def display(showquery, conn):
    table = fetch(showquery, conn)
    print("{0:20}{1:20}{2:20}".format("branch_name","branch_city","assets"))
    for (branch_name, branch_city, assets) in table:
        print("{0:20}{1:20}{2:<20}".format(branch_name, branch_city, assets))
        
    # fetch(showquery, conn)
    
    
    


In [9]:
conn = mysql.connector.connect(host=config.HOST,
                               user=config.USER, 
                               password=config.PASS)

print(conn.is_connected())

batch_execute_ddl(conn, 'bankdb.sql')

True
number of rows affected by statement drop database bankdb: 6
number of rows affected by statement create database if not exists bankdb: 1
number of rows affected by statement USE bankdb: 0
number of rows affected by statement CREATE TABLE branch (
  branch_name varchar(255) PRIMARY KEY,
  branch_city varchar(255),
  assets float
): 0
number of rows affected by statement CREATE TABLE customer (
  customer_name varchar(255) PRIMARY KEY,
  customer_street varchar(255),
  customer_city varchar(255)
): 0
number of rows affected by statement CREATE TABLE account (
  account_number varchar(255) PRIMARY KEY,
  branch_name varchar(255),
  balance float CHECK(balance >= 0),
  FOREIGN KEY (branch_name) REFERENCES branch (branch_name)
): 0
number of rows affected by statement CREATE TABLE depositor (
  customer_name varchar(255),
  account_number varchar(255),
  FOREIGN KEY (account_number) REFERENCES account (account_number),
  FOREIGN KEY (customer_name) REFERENCES customer (customer_name),

# Question 1
In a Jupyter Notebook, write code either in Python, Java or C to connect to your AWS RDS
instance and insert the data from each of the CSV files into the RDS database that you created on AWS on
Activity 1. Display the number of rows affected for each table (1 cumulative count for all inserts per table).
(Hint: If your data types do not match the given data, you may use the following SQL statement to update
your data types: ALTER TABLE table_name MODIFY column_name NEW_DATA_TYPE;)

In [10]:
path = "csvs/"
tables = os.listdir("csvs/") 

keys = [os.path.splitext(table)[0] for table in tables]
print(keys)

['depositor', 'loan', 'branch', 'customer', 'borrower', 'account']


In [11]:
d = dict.fromkeys(keys)
# print(d)
for key in keys:
    with open((path+key+'.csv'), newline='') as file:
        reader = csv.reader(file, delimiter=',')
        #ignore the column name
        next(reader)
        
        rows = []
        for row in reader:
            rows.append(tuple(row))
            
        d[key] = rows

print(d.keys())

                

dict_keys(['depositor', 'loan', 'branch', 'customer', 'borrower', 'account'])


In [12]:
customer_insert = """insert into customer
                  (customer_name, customer_street, customer_city)
                  values(%s, %s, %s);"""

branch_insert =  """insert into branch
                 (branch_name, branch_city, assets) 
                 values(%s, %s, %s);"""

account_insert = """insert into account
                    (account_number, branch_name, balance)
                    values(%s, %s, %s);"""

depositor_insert = """insert into depositor
                    (customer_name, account_number)
                    values(%s, %s);"""

loan_insert = """insert into loan
                (loan_number, branch_name, amount)
                values(%s, %s, %s);"""

borrower_insert = """insert into borrower
                    (customer_name, loan_number)
                    values(%s, %s);"""


In [13]:
conn = mysql.connector.connect(host=config.HOST,
                               user=config.USER, 
                               password=config.PASS,
                               database=config.MYDB)


In [14]:
insert_data(branch_insert, d['branch'], conn)


Rows affected:  9 
Data inserted successfully


In [15]:
insert_data(customer_insert, d['customer'], conn)

Rows affected:  15 
Data inserted successfully


In [16]:
insert_data(account_insert, d['account'], conn)

Rows affected:  9 
Data inserted successfully


In [17]:
insert_data(depositor_insert, d['depositor'], conn)

Rows affected:  10 
Data inserted successfully


In [18]:
insert_data(loan_insert, d['loan'], conn)

Rows affected:  9 
Data inserted successfully


In [19]:
insert_data(borrower_insert, d['borrower'], conn)


Rows affected:  10 
Data inserted successfully


# Question 2
Write code to issue queries on your database and display your results for the following scenarios. **Make sure to print out a tabulation of your branch table ordered by assets in descending order to show the data before and after the changes.**


A branch of the bank is considered a large branch if it has assets >= 3 million dollars. Imagine that COVID19 affected the bank terribly and the branches with assets >= 3 million dollars had to liquidate 12.5% of
their assets to help the smaller branches (branches with assets < 3 million dollars).


### (a) Issue a query that returns the total amount to be liquidated

In [20]:
query = "select sum(assets*0.125) from branch where (assets >= 3000000);"

result = fetch(query, conn)[0][0]
print(result)

2337500.0


### (b) Issue an update query that reduces the assets for the large branches by the liquidation formula above.

### Before

In [21]:
showquery = "select * from branch order by assets desc;"
display(showquery, conn)

branch_name         branch_city         assets              
Round Hill          Horseneck           8000000.0           
Brighton            Brooklyn            7000000.0           
North Town          Rye                 3700000.0           
Redwood             Palo Alto           2100000.0           
Perryridge          Horseneck           1700000.0           
Downtown            Brooklyn            900000.0            
Central             Rye                 400280.0            
Mianus              Horseneck           400200.0            
Pownal              Bennington          400000.0            


### After

In [22]:
query = "update branch set assets = assets*(1-0.125) where (assets >= 3000000);"

update(query, conn)

showquery = "select * from branch order by assets desc;"

display(showquery, conn)


branch_name         branch_city         assets              
Round Hill          Horseneck           7000000.0           
Brighton            Brooklyn            6125000.0           
North Town          Rye                 3237500.0           
Redwood             Palo Alto           2100000.0           
Perryridge          Horseneck           1700000.0           
Downtown            Brooklyn            900000.0            
Central             Rye                 400280.0            
Mianus              Horseneck           400200.0            
Pownal              Bennington          400000.0            


### (c) Issue an update query that divides equally the total amount liquidated among the small branches.

### Before

In [23]:
showquery = "select * from branch order by assets desc;"

display(showquery, conn)

branch_name         branch_city         assets              
Round Hill          Horseneck           7000000.0           
Brighton            Brooklyn            6125000.0           
North Town          Rye                 3237500.0           
Redwood             Palo Alto           2100000.0           
Perryridge          Horseneck           1700000.0           
Downtown            Brooklyn            900000.0            
Central             Rye                 400280.0            
Mianus              Horseneck           400200.0            
Pownal              Bennington          400000.0            


### After

In [24]:
#-- (select sum(assets*0.125) as liquid from branch where assets >= 3000000) T,
query = """update branch,  
           (select count(*) as total from branch where assets < 3000000) T2
           set assets = ( %s / T2.total ) + assets
           where (assets < 3000000)
           """
        
try:
    cursor = conn.cursor()
    cursor.execute(query % result)

    conn.commit()
except mysql.connector.Error as error:
    print(error)
    conn.rollback()

showquery = "select * from branch order by assets desc;"

display(showquery, conn)

branch_name         branch_city         assets              
Round Hill          Horseneck           7000000.0           
Brighton            Brooklyn            6125000.0           
North Town          Rye                 3237500.0           
Redwood             Palo Alto           2489580.0           
Perryridge          Horseneck           2089580.0           
Downtown            Brooklyn            1289580.0           
Central             Rye                 789863.0            
Mianus              Horseneck           789783.0            
Pownal              Bennington          789583.0            


### (d) Display a tabulation showing the customer_name, customer_city, and account_number of all customers who are members of a small branch. Include the branch_name and assets in your results and order your results by assets in descending order

In [25]:
query = """select customer_name, customer_city, account_number, branch_name, assets from 
                (customer C 
                natural join
                depositor D)
                natural join
                (account A
                natural join 
                branch B)
                where assets < 3000000
                """
table = fetch(query, conn)
# print(table)
print("{0:20}{1:20}{2:20}{3:20}{4:20}".format("customer_name","customer_city","account_number", "branch_name", "assets"))
for (customer_name,customer_city,account_number, branch_name, assets) in table:
    # for value in entry:
        # print("{0:20}".format(value))
    print("{0:20}{1:20}{2:20}{3:20}{4:<20}".format(customer_name,customer_city,account_number, branch_name, assets))


customer_name       customer_city       account_number      branch_name         assets              
Majeris             Rye                 A-333               Central             789863.0            
Hayes               Harrison            A-101               Downtown            1289580.0           
Johnson             Palo Alto           A-101               Downtown            1289580.0           
Smith               Rye                 A-215               Mianus              789783.0            
Hayes               Harrison            A-102               Perryridge          2089580.0           
Johnson             Palo Alto           A-201               Perryridge          2089580.0           
Lindsay             Pittsfield          A-222               Redwood             2489580.0           


In [20]:
conn.close()